In [146]:
import hashlib
from importlib import reload

In [166]:
import shortest_route
from shortest_route.lib import elements
from shortest_route.lib import graph
from shortest_route.lib.graph import CityMapperGraph
from shortest_route.lib.elements import Node, Edge
reload(shortest_route)
from shortest_route.lib.graph import CityMapperGraph
from shortest_route.lib.elements import Node, Edge

### Load data into nodes and edges

In [167]:
nodes = []
edges = []

with open('../data/shortest_route/citymapper-coding-test-graph.dat', 'r') as f:
    data = f.readlines()
    
# Number of vertices
N = int(data[0].strip("\n"))
for i in range(1, N+1):
    nodes.append(Node(uid=int(data[i].strip("\n"))))

# Number of edges
E = int(data[N+1])
for i in range(N+2, E+1):
    src, dst, length = data[i].strip("\n").split(" ")
    e_id = hashlib.sha256("{}_{}".format(src,dst).encode()).hexdigest()
    edges.append(Edge(uid=e_id, src=Node(uid=int(src)), dst=Node(uid=int(dst)), length=int(length)))

### Graph class

In [169]:
g = CityMapperGraph()
# nodeTrue = Node(id=316319897)
# nodeFalse = Node(0)
# edgeTrue = Edge(id="1c0597617b180d1bbe34e89bdd5370d923428868554c7190274535614b806d33", 
#                 src=Node(id=316319897), 
#                 dst=Node(316319936), 
#                 length=121)
# edgeFalse = Edge(id="false", src=Node(uid=0), dst=Node(uid=1), length=0)
# srcNodeTrue, dstNodeTrue = edgeTrue.src, edgeTrue.dst

# g.add_nodes(1084140204)
# g.add_nodes([nodeTrue, srcNodeTrue, dstNodeTrue])
# g.add_edges(edgeTrue)


AssertionError: 

In [144]:
# list(g.find_nodes(['1084140204'])), # Passing an array of string
# list(g.find_nodes([1084140204])), # Passing an array of integer
list(g.find_nodes(nodeTrue)), # Passing in a Node

# Node(id=316319897) in g.nodes

False


([],)

In [145]:
isinstance(nodeTrue, Node)

True

### Unit Tests

In [ ]:
#########
# Nodes #
#########
pos_results = [
    list(g.find_nodes('1084140204')), # Passing a string
    list(g.find_nodes(['1084140204'])), # Passing an array of string
    list(g.find_nodes(1084140204)), # Passing an integer
    list(g.find_nodes([1084140204])), # Passing an array of integer
    list(g.find_nodes(nodeTrue)), # Passing in a Node
    list(g.find_nodes([nodeTrue])) # Passing in an array of Node
]

neg_results = [
    list(g.find_nodes('0')), # Passing a string
    list(g.find_nodes(['0'])), # Passing an array of string
    list(g.find_nodes(0)), # Passing an integer
    list(g.find_nodes([0])), # Passing an array of integer
    list(g.find_nodes(nodeFalse)), # Passing in a Node
    list(g.find_nodes([nodeFalse])) # Passing in an array of Node
]

N = len(pos_results)
M = len(neg_results)

assert all([len(pos_results[i]) == 1 for i in range(N)])
assert all([pos_results[i] for i in range(N)])
assert all([len(neg_results[i]) == 0 for i in range(M)])
assert all([not neg_results[i] for i in range(M)])

#########
# Edges #
#########

# pos_results = [
#     list(g.find_edges(edgeTrue.id)), # Passing a string
#     list(g.find_edges(edgeTrue)), # Passing a string
#     list(g.find_edges([edgeTrue.id])), # Passing a string
#     list(g.find_edges([edgeTrue])), # Passing a string
#     list(g.find_edges((srcNodeTrue, dstNodeTrue))), 
#     list(g.find_edges([(srcNodeTrue, dstNodeTrue)]))
# ]

# neg_results = [
#     list(g.find_edges(edgeFalse.id)), # Passing a string
#     list(g.find_edges(edgeFalse)), # Passing a string
#     list(g.find_edges([edgeFalse.id])), # Passing a string
#     list(g.find_edges([edgeFalse])), # Passing a string
#     list(g.find_edges((nodeTrue, nodeFalse))), 
#     list(g.find_edges([(nodeTrue, nodeFalse)])),
#     list(g.find_edges((nodeFalse, nodeTrue))), 
#     list(g.find_edges([(nodeFalse, nodeTrue)]))    
# ]

# N = len(pos_results)
# M = len(neg_results)

# assert all([len(pos_results[i]) == 1 for i in range(N)])
# assert all([pos_results[i] for i in range(N)])
# assert all([len(neg_results[i]) == 0 for i in range(M)])
# assert all([not neg_results[i] for i in range(M)])